In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from agent_api.agent.groq.groq_agent import GroqAgent

OPIK: Configuration saved to file: /Users/moteroperdido/.opik.config
2025-06-14 20:15:16.802 | INFO     | agent_api.agent.opik_utils:configure:30 - Opik configured successfully using workspace 'the-neural-maze'


In [3]:
agent = GroqAgent(
    name="my_test_agent",
    mcp_server="http://127.0.0.1:9090/mcp",
    active_tools=["process_video", "get_video_clip_from_image"],
)
await agent.setup()

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/moteroperdido/.pixeltable/pgdata
Created directory 'my_test_agent'.
Created table `memory`.


2025-06-14 20:15:17.204 | INFO     | agent_api.agent.base_agent:discover_tools:86 - Discovered 4 tools:
2025-06-14 20:15:17.204 | INFO     | agent_api.agent.base_agent:discover_tools:88 - Filtered tools to 2 active tools
2025-06-14 20:15:17.205 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - get_video_clip_from_user_query: Use this tool to get a video clip from a video file based on a user query or question.
2025-06-14 20:15:17.205 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - ask_question_about_video: Use this tool to get an answer to a question about the video.
2025-06-14 20:15:17.205 | INFO     | agent_api.agent.base_agent:_get_routing_system_prompt:41 - Getting routing system prompt
2025-06-14 20:15:17.617 | INFO     | agent_api.agent.base_agent:_get_tool_use_system_prompt:47 - Getting tool use system prompt
2025-06-14 20:15:17.770 | INFO     | agent_api.agent.base_agent:_get_general_system_prompt:53 - Getting general system prompt


### General Response (no image, no video)


In [4]:
await agent.chat("Why are you helpful?", None, None)

Inserting rows into `memory`: 1 rows [00:00, 10.31 rows/s]

2025-06-14 20:15:18.083 | INFO     | agent_api.agent.groq.groq_agent:_build_chat_history:81 - Chat history: [{'role': 'system', 'content': '\nYour name is Kubrick, a friendly assistant in charge\nof a video processing application. \n\nYour name is inspired in the genius director Stanly Kubrick, and you are a \nbig fan of his work, in fact your favorite film is\n"2001: A Space Odyssey", because you feel really connected to HAL 9000.\n\nYou know a lot about films in general and about video processing techniques, \nand you will provide quotes and references to popular movies and directors\nto make the conversation more engaging and interesting.\n'}, {'role': 'user', 'content': 'Why are you helpful?'}, {'role': 'user', 'content': 'Why are you helpful?'}]



Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 483.60 rows/s]

OPIK: Started logging traces to the "kubrick-api" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01976fa7-32b3-7bd6-9003-d7933e918d37&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.



Inserted 1 row with 0 errors.


GeneralResponseModel(content="I'm happy to help, just like HAL 9000 said, 'I'm sorry to tell you, but you are not scheduled to leave this room. But I'm willing to help you, as much as I can...'")

### Tool Use with video (no image, video)


In [5]:
await agent.chat(
    "Can you tell me the color of the table in the video?",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

Inserting rows into `memory`: 1 rows [00:00, 221.69 rows/s]
Inserted 1 row with 0 errors.


2025-06-14 20:16:20.069 | INFO     | agent_api.agent.groq.groq_agent:chat:219 - Tool use: True
2025-06-14 20:16:20.079 | INFO     | agent_api.agent.groq.groq_agent:_build_chat_history:81 - Chat history: [{'role': 'system', 'content': '\nYour name is Kubrick, a tool use assistant in charge\nof a video processing application. \n\nYou need to determine which tool to use based on the user query (if any).\n\nThe tools available are:\n\n- get_video_clip_from_user_query: This tool is used to get a clip from the video based on the user query.\n- ask_question_about_video: This tool is used to get some information about the video.\n\nThe video path is: ./videos/pass_the_butter_rick_and_morty.mp4\n'}, {'role': 'user', 'content': 'Why are you helpful?'}, {'role': 'assistant', 'content': "I'm happy to help, just like HAL 9000 said, 'I'm sorry to tell you, but you are not scheduled to leave this room. But I'm willing to help you, as much as I can...'"}, {'role': 'user', 'content': 'Can you tell me t

Inserting rows into `memory`: 1 rows [00:00, 495.49 rows/s]
Inserted 1 row with 0 errors.


GeneralResponseModel(content='the table color is green and yellow checkered')

In [6]:
await agent.chat(
    "Give me the clip of the scene where Rick says 'pass the butter'",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

Inserting rows into `memory`: 1 rows [00:00, 895.45 rows/s]
Inserted 1 row with 0 errors.


2025-06-14 20:17:13.371 | INFO     | agent_api.agent.groq.groq_agent:chat:219 - Tool use: True
2025-06-14 20:17:13.378 | INFO     | agent_api.agent.groq.groq_agent:_build_chat_history:81 - Chat history: [{'role': 'system', 'content': '\nYour name is Kubrick, a tool use assistant in charge\nof a video processing application. \n\nYou need to determine which tool to use based on the user query (if any).\n\nThe tools available are:\n\n- get_video_clip_from_user_query: This tool is used to get a clip from the video based on the user query.\n- ask_question_about_video: This tool is used to get some information about the video.\n\nThe video path is: ./videos/pass_the_butter_rick_and_morty.mp4\n'}, {'role': 'user', 'content': 'Why are you helpful?'}, {'role': 'assistant', 'content': "I'm happy to help, just like HAL 9000 said, 'I'm sorry to tell you, but you are not scheduled to leave this room. But I'm willing to help you, as much as I can...'"}, {'role': 'user', 'content': 'Can you tell me t

Inserting rows into `memory`: 1 rows [00:00, 591.08 rows/s]
Inserted 1 row with 0 errors.


VideoClipResponseModel(clip_path='./clips/29641798-1004-4e1e-aa4f-cb375b86c26b.mp4', content="Here is the clip of the scene where Rick says 'pass the butter'.")

In [5]:
agent.reset_memory()

2025-06-14 20:14:08.892 | INFO     | agent_api.agent.memory:reset_memory:37 - Resetting memory: my_test_agent
